<a href="https://colab.research.google.com/github/Rnlcksgdkd/Project_AI/blob/ando/ando/test_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import json
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os, sys
from google.colab import drive
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle
import re

!pip install konlpy
from konlpy.tag import Okt


In [2]:
!git clone https://github.com/Rnlcksgdkd/Project_AI

Cloning into 'Project_AI'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 343 (delta 33), reused 23 (delta 21), pack-reused 298
Receiving objects: 100% (343/343), 123.02 MiB | 28.00 MiB/s, done.
Resolving deltas: 100% (158/158), done.


In [3]:
%cd /content/Project_AI 
!git checkout ando


/content/Project_AI
Checking out files: 100% (40/40), done.
Branch 'ando' set up to track remote branch 'ando' from 'origin'.
Switched to a new branch 'ando'


In [ ]:
import sys
DATA_PATH = "/content/Project_AI/ando/NaverMovie/"

sys.path.append(DATA_PATH)
train_df = pd.read_csv('Naver_MR_train.csv')
train_input = np.load("Naver_MR_train_input.npy")
train_label = np.load("Naver_MR_train_label.npy")
test_input = np.load("Naver_MR_test_input.npy")
test_label = np.load("Naver_MR_test_label.npy")

prepro_configs = json.load(open(DATA_PATH + "data_config.json" , 'r'))

train_input.shape , train_label.shape , test_input.shape , test_label.shape ,   prepro_configs['vocab_size']

((149995, 15), (149995,), (149995, 15), (149995,), 38771)

In [4]:

# CNN 모델 클래스로 정의
from tensorflow.keras import layers

class CNNClassifier(tf.keras.Model):

  def __init__(self , **kargs):
    
    super(CNNClassifier , self).__init__(name = kargs['model_name'])

    self.embedding = layers.Embedding(input_dim = kargs['vocab_size'] , output_dim = kargs['embedding_size'])

    self.conv_list = [layers.Conv1D(filters = kargs['num_filters'] , kernel_size = kernel_size  , 
                                    padding = 'valid' , activation = 'relu', kernel_constraint = tf.keras.constraints.MaxNorm(max_value = 3.) )
                                   for kernel_size in [3,4,5] ]

    self.pooling  = layers.GlobalMaxPooling1D()
    self.dropout = layers.Dropout(kargs['dropout_rate'])
    self.fc1 = layers.Dense(kargs['hidden_dimension'] , activation = 'relu' ,  kernel_constraint = tf.keras.constraints.MaxNorm(max_value = 3.))
    self.fc2 = layers.Dense(kargs['ouput_dimension'] , activation = 'sigmoid' ,  kernel_constraint = tf.keras.constraints.MaxNorm(max_value = 3.))
  

  def call(self,x):

    x = self.embedding(x)
    x = self.dropout(x)
    x = tf.concat([self.pooling(conv(x)) for conv in self.conv_list] , axis = 1)
    x = self.fc1(x)
    x = self.fc2(x)
    
    return x



In [7]:
# 신경망 생성 및 가중치 로드
def build_Sentiment_model():

  model_name = 'CNN'
  MAX_LEN = 15

  kargs = {'model_name' : model_name , 'vocab_size' : 38771 , 'embedding_size' : 256 , 'num_filters' : 150 , 'dropout_rate' : 0.5 , 'hidden_dimension': 512 ,'ouput_dimension' : 1}

  model = CNNClassifier(**kargs)
  model.compile(optimizer = tf.keras.optimizers.Adam(1e-4) , 
              loss = 'binary_crossentropy' ,
              metrics = ['accuracy'])
  
  dummy_input = np.zeros((1,15))
  dummy_label = np.zeros((1,1))
  model.fit(dummy_input , dummy_label , batch_size= 1 , epochs = 1)
  model.load_weights('/content/Project_AI/ando/NaverMovie/Models/CNN_2_weights.h5')

  return model
  

In [26]:


class predict_Sentiment():
  
  def __init__(self):

    self.model = build_Sentiment_model()

    ## 전처리 필요한 것들 로드

    # 불용어 사전
    self.stop_words = ['은', '는' , '이' , '가' , '하', '아', '것' , '들', '의', '있' , '되' , '수' , '보' , '주' , '등' ,'한' ]

    # 토크나이저 로드
    with open('/content/Project_AI/ando/NaverMovie/tokenizer.pickle', 'rb') as handle:
      self.tokenizer = pickle.load(handle)

    # Okt 형태소 추출기 생성
    self.okt = Okt()


    ## 예측 신경망 생성 및 로드

  # 전처리 함수
  def preprocessing(self , text , limit_words = 15 , print_option = False ):

    if print_option : print("원본".ljust(15 , ' ') + ": " , text)
    
    text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]" , "" , text)
    if print_option : print("정규표현식 처리".ljust(15 , ' ') + ": " , text)

    text = self.okt.morphs(text , stem = True)
    if print_option : print("okt 형태소 추출".ljust(15 , ' ') + ": " , text)


    text = [word for word in text[:limit_words]]
    if print_option : print("문장 길이 조절".ljust(15 , ' ') + ": " , text)

    text = [token for token in text if not token in self.stop_words]
    if print_option : 
      print("불용어 처리".ljust(15 , ' ') + ": " , text)
      print(" ")
      print(" ")

    return text

  # 감성 예측
  def pred_Sentiment(self , input):

    print("입력 값 : " , input)
    print(" ")
    input = self.preprocessing(input , 15 , False)
    input = self.tokenizer.texts_to_sequences([input])
    input = pad_sequences(input , maxlen = 15 , padding = 'post')

    pred = round(self.model.predict(input)[0][0]*100 , 2)
    if pred >= 50 : print("긍정 ({} %)".format(pred))
    else : print("부정 ({} %)".format(pred))
    
    return pred
    

    

In [22]:
pred = predict_Sentiment()

pred.pred_Sentiment("안녕하세요")

1/1 [==============================] - 2s 2s/step - loss: 0.6849 - accuracy: 1.0000
입력 값 :  안녕하세요
 
원본             :  안녕하세요
정규표현식 처리       :  안녕하세요
okt 형태소 추출     :  ['안녕하다']
문장 길이 조절       :  ['안녕하다']
불용어 처리         :  ['안녕하다']
 
 
부정 (14.85 %)


14.85

In [23]:
pred.pred_Sentiment("이 영화 정말 재미있습니다 추천드립니다")

입력 값 :  이 영화 정말 재미있습니다 추천드립니다
 
원본             :  이 영화 정말 재미있습니다 추천드립니다
정규표현식 처리       :  이 영화 정말 재미있습니다 추천드립니다
okt 형태소 추출     :  ['이', '영화', '정말', '재미있다', '추천', '드리다']
문장 길이 조절       :  ['이', '영화', '정말', '재미있다', '추천', '드리다']
불용어 처리         :  ['영화', '정말', '재미있다', '추천', '드리다']
 
 
긍정 (95.96 %)


95.96

In [24]:
pred.pred_Sentiment("기레기 놈들 제발 이딴 쓰레기 같은 기사 좀 쓰지마")


입력 값 :  기레기 놈들 제발 이딴 쓰레기 같은 기사 좀 쓰지마
 
원본             :  기레기 놈들 제발 이딴 쓰레기 같은 기사 좀 쓰지마
정규표현식 처리       :  기레기 놈들 제발 이딴 쓰레기 같은 기사 좀 쓰지마
okt 형태소 추출     :  ['기레기', '놈', '들', '제발', '이딴', '쓰레기', '같다', '기사', '좀', '쓰다']
문장 길이 조절       :  ['기레기', '놈', '들', '제발', '이딴', '쓰레기', '같다', '기사', '좀', '쓰다']
불용어 처리         :  ['기레기', '놈', '제발', '이딴', '쓰레기', '같다', '기사', '좀', '쓰다']
 
 
부정 (0.39 %)


0.39

In [25]:
pred.pred_Sentiment("뭐하냐 진짜")

입력 값 :  뭐하냐 진짜
 
원본             :  뭐하냐 진짜
정규표현식 처리       :  뭐하냐 진짜
okt 형태소 추출     :  ['뭐', '하다', '진짜']
문장 길이 조절       :  ['뭐', '하다', '진짜']
불용어 처리         :  ['뭐', '하다', '진짜']
 
 
부정 (9.17 %)


9.17

In [27]:
pred.pred_Sentiment("와 정말 대단하신 분이네요 존경스럽습니다")

입력 값 :  와 정말 대단하신 분이네요 존경스럽습니다
 
원본             :  와 정말 대단하신 분이네요 존경스럽습니다
정규표현식 처리       :  와 정말 대단하신 분이네요 존경스럽습니다
okt 형태소 추출     :  ['오다', '정말', '대단하다', '불다', '존경', '스럽다']
문장 길이 조절       :  ['오다', '정말', '대단하다', '불다', '존경', '스럽다']
불용어 처리         :  ['오다', '정말', '대단하다', '불다', '존경', '스럽다']
 
 
긍정 (87.64 %)


87.64

In [ ]:

Input = "미쳤냐?????? 진짜 정신 나갔네"

pred.show_predict(Input)


입력 값 :  미쳤냐?????? 진짜 정신 나갔네
 
원본             :  미쳤냐?????? 진짜 정신 나갔네
정규표현식 처리       :  미쳤냐 진짜 정신 나갔네
okt 형태소 추출     :  ['미치다', '진짜', '정신', '나가다']
문장 길이 조절       :  ['미치다', '진짜', '정신', '나가다']
불용어 처리         :  ['미치다', '진짜', '정신', '나가다']
 
 
[[254, 16, 487, 568]]
부정 (8.73 %)
